### Introduction to graphs and tf.function
> One of the advantage of tf compared to other deep learning frameworks is because tf is much more flexible when it comes to deployment. 
<br> </br>
>>  So, why are we talking about deployment in this specific turtorial?<br>
Because in this turtorial we will be learning tf.Graph which enable tf models to run device without python interpretator. 


### What are graphs?
In the previous three guides, you ran TensorFlow eagerly. This means TensorFlow operations are executed by Python, operation by operation, and returning results back to Python.

While eager execution has several unique advantages, graph execution enables portability outside Python and tends to offer better performance. Graph execution means that tensor computations are executed as a TensorFlow graph, sometimes referred to as a tf.Graph or simply a "graph."

Graphs are data structures that contain a set of tf.Operation objects, which represent units of computation; and tf.Tensor objects, which represent the units of data that flow between operations. They are defined in a tf.Graph context. Since these graphs are data structures, they can be saved, run, and restored all without the original Python code.

>In short, graphs are extremely useful and let your TensorFlow run fast, run in parallel, and run efficiently on multiple devices. <br>

However, you still want to define your machine learning models (or other computations) in Python for convenience, and then automatically construct graphs when you need them..

In [28]:
import tensorflow as tf 
import timeit
from datetime import datetime
from time import time

To create and run graph in Tensorflow we use **tf.funtion**,either as a direct call or as *decorator*.
> decorator: In python decorator are used to add more meaning to the existing function. It can be thought as passing a certain function as parameter to decorator funtion.
<br>
decorator_fnc(normal_fnc), here normal fnc are feeded as paramter to decorator fucntion.  

In [2]:
## Define a Python function




def reg_fnc(x,y):
    return tf.multiply(x,y)

## Creating a funtion using tf.function or creating graphs in simple words
graph_fnc=tf.function(reg_fnc)

### Creating some tensors 

x=tf.constant([[1,2,3]])
y=tf.constant([[5,6,7]])

print("Output from regular function : {}".format(reg_fnc(x,y)))
print("Output from graph function : {}".format(graph_fnc(x,y)))


Output from regular function : [[ 5 12 21]]
Output from graph function : [[ 5 12 21]]


2022-10-03 14:42:08.047771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 14:42:08.066188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 14:42:08.066557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 14:42:08.067338: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

On the outside, a Function looks like a regular function you write using TensorFlow operations. Underneath, however, it is very different. A Function encapsulates several tf.Graphs behind one API (learn more in the Polymorphism section). That is how a Function is able to give you the benefits of graph execution, like speed and deployability (refer to The benefits of graphs above).

> tf.function applies to a function and all other functions it calls:

In [3]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

### Polymorphism: one Function, many graphs
> A tf.Graph is specialized to a specific type of inputs (for example, tensors with a specific dtype or objects with the same id()).
Each time you invoke a Function with a set of arguments that can't be handled by any of its existing graphs (such as arguments with new dtypes or incompatible shapes), Function creates a new tf.Graph specialized to those new arguments. The type specification of a tf.Graph's inputs is known as its input signature or just a signature. For more information regarding when a new tf.Graph is generated and how that can be controlled, go to the Rules of tracing section of the Better performance with tf.function guide.
The Function stores the tf.Graph corresponding to that signature in a ConcreteFunction. A ConcreteFunction is a wrapper around a tf.Graph.

>below functions are copied from turtorial of Tensorflow official websites.

In [4]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


In [6]:
print(my_relu(tf.constant(3.3))) ## it will have same graph as they have the same signatures

tf.Tensor(3.3, shape=(), dtype=float32)


### Graph exucution vs eager execution
> eager execution refer to the execution of those tf model which interpreter is python. (all execution with out using tf.function are eagerly executed.)
> <br> by default, when tf.function is used Function are executed as graph. But it can also be eagerly executed by using **tf.config.run_functions_eagerly(True)**.

In [17]:
@tf.function
def get_MSE(y_true,y_pred):
    sq_diff=tf.pow(y_true-y_pred,2)
    return tf.reduce_mean(sq_diff)

In [18]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

tf.Tensor([6 8 9 3 9], shape=(5,), dtype=int32)
tf.Tensor([4 4 7 1 3], shape=(5,), dtype=int32)


In [19]:
get_MSE(y_true,y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=12>

>To verify that your Function's graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with tf.config.run_functions_eagerly(True). This is a switch that turns off Function's ability to create and run graphs, instead of executing the code normally.

In [22]:
tf.config.run_functions_eagerly(True)
print(get_MSE(y_true, y_pred)) 
## It is essential to set it back when we are done. 
tf.config.run_functions_eagerly(False)

tf.Tensor(12, shape=(), dtype=int32)


### Function tracing
> "tracing" means to run a Python function and "record" its **TensorFlow operations** in a graph.
<br>
Let us see example of how tracing works in Graph execution and compare it to the eager execution.


In [25]:
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)
  

In [26]:
## Calling the above function two times.
print(get_MSE(y_true, y_pred)) 
print(get_MSE(y_true, y_pred)) 

Calculating MSE!
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)


In the above function we noticed while values are returned two time, but the print function is execution only once. Why this happens? 
<br>
> To explain, the print statement is executed when Function runs the original code in order to create the graph in a process known as "tracing" (refer to the Tracing section of the tf.function guide. Tracing captures the TensorFlow operations into a graph, and print is not captured in the graph. That graph is then executed for all two calls without ever running the Python code again.


### Speed comparision of eager execution vs graph execution


In [40]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)


def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [41]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000))

Eager execution: 0.9339566340058809


In [42]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000))

Graph execution: 0.8861332800006494


### Performance and trade-offs
Graphs can speed up your code, but the process of creating them has some overhead. For some functions, the creation of the graph takes more time than the execution of the graph. This investment is usually quickly paid back with the performance boost of subsequent executions, but it's important to be aware that the first few steps of any large model training can be slower due to tracing.

No matter how large your model, you want to avoid tracing frequently. ***The tf.function guide discusses how to set input specifications and use tensor arguments to avoid retracing in the Controlling retracing section. If you find you are getting unusually poor performance, it's a good idea to check if you are retracing accidentally.***

> credit : Tensorflow

### learn more about tf.function best practises and Non-strict execution later.